# Objective
------------
This notebook is to illustrate the different steps that are in the data curation workflow:
- Data Acquistion
- Data Cleaning and Homogenization
- Feature Engineering

# Libraries

In [1]:
from SourceCode.DataCuration.data_acquisition import unzip_archives, download_openmeteo_for_cities
from SourceCode.DataCuration.data_cleaning_homogenization import (
    homogenize_metro_dataset, homogenize_capital_dataset, 
    generate_stations_dataframe,
    generate_bikes_dataframe, 
    generate_trips_dataframe,
    generate_demographics_dataframe,
    generate_weather_dataframe,
    map_trips_foreign_keys_to_uuids,
    map_stations_foreign_keys_to_uuids
)
from SourceCode.DataCuration.feature_engineering import generate_trips_feature_engineering

In [2]:
import pandas as pd
import glob
import os

# Data Acquisition

The data in the Raw data folder was obtained from: 

- Metro: https://bikeshare.metro.net/about/data/
- Capital: https://s3.amazonaws.com/capitalbikeshare-data/index.html 

In [7]:
# stablish folders
raw_data_folder = "./RawData"
temp_data_folder = "./TempData"

In [4]:
unzip_archives(raw_data_folder, ["MetroBike", "CapitalBike"])

# Expected result:
# All files are unzipped with the corresponding csv files

All files extracted from './RawData/MetroBike/la_metro_gbfs_trips_Q1_2017-2.zip' to './RawData/MetroBike'.
All files extracted from './RawData/MetroBike/la_metro_gbfs_trips_Q2_2017.csv.zip' to './RawData/MetroBike'.
All files extracted from './RawData/MetroBike/metro-bike-share-trips-2016-q4.csv.zip' to './RawData/MetroBike'.
All files extracted from './RawData/MetroBike/metro-bike-share-trips-2017-q3.csv.zip' to './RawData/MetroBike'.
All files extracted from './RawData/MetroBike/metro-bike-share-trips-2017-q4-v2.csv.zip' to './RawData/MetroBike'.
All files extracted from './RawData/MetroBike/metro-bike-share-trips-2018-q1.csv.zip' to './RawData/MetroBike'.
All files extracted from './RawData/MetroBike/metro-bike-share-trips-2018-q2.csv.zip' to './RawData/MetroBike'.
All files extracted from './RawData/MetroBike/metro-bike-share-trips-2018-q3.csv.zip' to './RawData/MetroBike'.
All files extracted from './RawData/MetroBike/metro-bike-share-trips-2018-q4.csv.zip' to './RawData/MetroBike

In [5]:
start_date = "2016-01-01"
end_date = "2025-10-01"

download_openmeteo_for_cities(raw_data_folder, start_date, end_date)

# Expected result:
# weather datasets for the cities of Los Angeles and Washington D.C.

LosAngeles
Coordinates: 34.05975341796875°N -118.23750305175781°E
Elevation: 92.0 m asl
Timezone difference to GMT+0: -28800s

------------------------------------------------------------
WashingtonDC
Coordinates: 38.91036605834961°N -77.072509765625°E
Elevation: 56.0 m asl
Timezone difference to GMT+0: -18000s

------------------------------------------------------------


In [6]:
# Demographic data will be downloaded based on the zip codes of the stations
# This is too narrow down only the data that is requiered

# Data Cleaning and Homogenization

## Weather

In [ ]:
df_weather = generate_weather_dataframe(raw_data_folder)
df_weather

# Expected result:
# All weather data combined into a single dataframe

,weather_uuid,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,...,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,soil_temperature_0_to_7cm,soil_moisture_0_to_7cm,soil_temperature_7_to_28cm,soil_moisture_7_to_28cm,wind_speed_100m,city
0,dc857a0b-fac2-4462-aec1-c66d9f511ff5,2016-01-01 00:00:00,5.4445,39.059063,-7.405500,0.953630,0.0,0.0,0.0,0.0,...,0.0,0.048416,0.549567,10.028439,5.844500,0.141,10.494499,0.161,21.434364,Los Angeles
1,98062b7a-c7c5-44a2-ab10-5ea18d1d64a5,2016-01-01 00:00:00,5.7085,59.195580,-1.641500,1.653484,0.0,0.0,0.0,0.0,...,97.0,0.032821,0.374834,11.592894,8.158501,0.477,10.658501,0.484,19.083395,Washington D.C.
2,405fb832-c0f1-489c-b096-4c68f91bb6f7,2016-01-01 01:00:00,4.8445,38.426790,-8.155500,0.265038,0.0,0.0,0.0,0.0,...,0.0,0.048352,0.532551,10.163227,5.344500,0.141,10.344500,0.161,21.120682,Los Angeles
3,916d13b6-1119-42d2-886f-150a08082270,2016-01-01 01:00:00,5.3085,58.876510,-2.091500,1.208356,0.0,0.0,0.0,0.0,...,90.0,0.032249,0.367427,11.480557,7.908500,0.476,10.508500,0.484,19.134262,Washington D.C.
4,b09687aa-7c9b-4ede-b76c-a5a9866d8904,2016-01-01 02:00:00,4.3445,37.976845,-8.755501,-0.122156,0.0,0.0,0.0,0.0,...,0.0,0.042747,0.518008,9.028754,4.844500,0.141,10.144500,0.161,21.971800,Los Angeles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170971,e85e24d9-c439-46e4-8cd4-9211e2beddbb,2025-10-01 22:00:00,15.7195,71.688810,10.619500,15.422153,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.506085,3.381893,16.769500,0.275,19.869501,0.329,11.225132,Washington D.C.
170972,32499b14-adf0-4181-8538-19a93f27ef4b,2025-10-01 23:00:00,18.7205,82.190895,15.620500,20.440535,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.384691,1.310420,22.920502,0.077,24.920502,0.144,1.938659,Los Angeles
170973,1f6a55d3-2954-41d0-94b8-047e55c5e9b1,2025-10-01 23:00:00,15.0695,74.244484,10.519500,14.882826,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.441621,2.421652,15.969500,0.275,19.769500,0.329,11.052058,Washington D.C.
170974,0f96723d-aa43-4aff-bf7d-c81ec5a8ae63,2025-10-02 00:00:00,18.3205,83.469440,15.470500,19.831667,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.348257,2.340000,22.020500,0.077,24.820500,0.144,3.758510,Los Angeles


## Bike datasets homogenization

In [ ]:
# homogenize to have common keys and format
df_metro = homogenize_metro_dataset(raw_data_folder)
df_capital = homogenize_capital_dataset(raw_data_folder)

# Expected result:
# Both datasets homogenized with common columns and formats

la_metro_gbfs_trips_Q1_2017.csv
la_metro_gbfs_trips_Q2_2017.csv
metro-bike-share-trips-2016-q4.csv
metro-bike-share-trips-2017-q3.csv
metro-bike-share-trips-2017-q4-v2.csv
metro-bike-share-trips-2018-q1.csv
metro-bike-share-trips-2018-q2.csv
metro-bike-share-trips-2018-q3.csv
metro-bike-share-trips-2018-q4.csv
metro-bike-share-trips-2019-q1.csv
metro-bike-share-trips-2019-q2.csv
metro-bike-share-trips-2019-q3.csv
metro-bike-share-trips-2019-q4.csv


D:\Masters\Data Curation\data_curation\./SourceCode\DataCuration\data_cleaning_homologation.py:46: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metro_segment = pd.read_csv(csv_file_path)


metro-bike-share-trips-2020-q1.csv
metro-trips-2020-q2-v2.csv
metro-trips-2020-q3.csv
metro-trips-2020-q4.csv
metro-trips-2021-q1.csv
metro-trips-2021-q2.csv


D:\Masters\Data Curation\data_curation\./SourceCode\DataCuration\data_cleaning_homologation.py:46: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metro_segment = pd.read_csv(csv_file_path)


metro-trips-2021-q3.csv
metro-trips-2021-q4.csv
metro-trips-2022-q1.csv
metro-trips-2022-q2.csv
metro-trips-2022-q3.csv
metro-trips-2022-q4.csv
metro-trips-2023-q1.csv


...

metro-trips-2023-q4.csv


D:\Masters\Data Curation\data_curation\./SourceCode\DataCuration\data_cleaning_homologation.py:46: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_metro_segment = pd.read_csv(csv_file_path)


metro-trips-2024-q1.csv
metro-trips-2024-q2.csv
metro-trips-2024-q3.csv
metro-trips-2024-q4.csv
metro-trips-2025-q1.csv
metro-trips-2025-q2.csv
MetroBikeShare_2016_Q3_trips.csv
2016Q1-capitalbikeshare-tripdata.csv
2016Q2-capitalbikeshare-tripdata.csv
2016Q3-capitalbikeshare-tripdata.csv
2016Q4-capitalbikeshare-tripdata.csv
2017Q1-capitalbikeshare-tripdata.csv
2017Q2-capitalbikeshare-tripdata.csv
2017Q3-capitalbikeshare-tripdata.csv
2017Q4-capitalbikeshare-tripdata.csv
201801_capitalbikeshare_tripdata.csv
201803-capitalbikeshare-tripdata.csv
201804-capitalbikeshare-tripdata.csv
201805-capitalbikeshare-tripdata.csv
201806-capitalbikeshare-tripdata.csv
201807-capitalbikeshare-tripdata.csv
201808-capitalbikeshare-tripdata.csv
201809-capitalbikeshare-tripdata.csv
201810-capitalbikeshare-tripdata.csv
201811-capitalbikeshare-tripdata.csv
201812-capitalbikeshare-tripdata.csv
201901-capitalbikeshare-tripdata.csv
201902-capitalbikeshare-tripdata.csv
201903-capitalbikeshare-tripdata.csv
201904-ca

D:\Masters\Data Curation\data_curation\./SourceCode\DataCuration\data_cleaning_homologation.py:87: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_capital_segment = pd.read_csv(csv_file_path)


202103-capitalbikeshare-tripdata.csv
202104-capitalbikeshare-tripdata.csv
202105-capitalbikeshare-tripdata.csv
202106-capitalbikeshare-tripdata.csv
202107-capitalbikeshare-tripdata.csv
202108-capitalbikeshare-tripdata.csv
202109-capitalbikeshare-tripdata.csv
202110-capitalbikeshare-tripdata.csv
202111-capitalbikeshare-tripdata.csv
202112-capitalbikeshare-tripdata.csv
202201-capitalbikeshare-tripdata.csv
202202-capitalbikeshare-tripdata.csv
202203-capitalbikeshare-tripdata.csv
202205-capitalbikeshare-tripdata.csv
202206-capitalbikeshare-tripdata.csv
202207-capitalbikeshare-tripdata.csv
202208-capitalbikeshare-tripdata.csv
202209-capitalbikeshare-tripdata.csv
202210-capitalbikeshare-tripdata.csv
202211-capitalbikeshare-tripdata.csv
202212-capitalbikeshare-tripdata.csv
202301-capitalbikeshare-tripdata.csv
202302-captialbikeshare-tripdata.csv
202303-capitalbikeshare-tripdata.csv
202304-capitalbikeshare-tripdata.csv
202305-capitalbikeshare-tripdata.csv
202306-capitalbikeshare-tripdata.csv
2

In [ ]:
# Homogenized dataset for Metro
df_metro

,trip_id,duration,start_time,end_time,start_station_id,start_lat,start_lon,end_station_id,end_lat,end_lon,bike_id,plan_duration,trip_route_category,member_type,bike_type,start_station_name,end_station_name
0,17059131,8,1/1/2017 0:15,1/1/2017 0:23,3030,34.051941,-118.243530,3029,34.048851,-118.246422,6220,30.0,One Way,Monthly Pass,NaN,NaN,NaN
1,17059130,12,1/1/2017 0:24,1/1/2017 0:36,3028,34.058319,-118.246094,3028,34.058319,-118.246094,6351,0.0,Round Trip,Walk-up,NaN,NaN,NaN
2,17059129,17,1/1/2017 0:28,1/1/2017 0:45,3027,34.049980,-118.247162,3018,34.043732,-118.260139,5836,0.0,One Way,Walk-up,NaN,NaN,NaN
3,17059128,5,1/1/2017 0:38,1/1/2017 0:43,3007,34.050480,-118.254593,3031,34.044701,-118.252441,6142,30.0,One Way,Monthly Pass,NaN,NaN,NaN
4,17059127,5,1/1/2017 0:38,1/1/2017 0:43,3007,34.050480,-118.254593,3031,34.044701,-118.252441,6135,30.0,One Way,Monthly Pass,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2830776,434777414,6,9/9/2024 9:58,9/9/2024 10:04,4562,34.026970,-118.428207,4645,34.023411,-118.429008,24833,30.0,One Way,Monthly Pass,electric,NaN,NaN
2830777,434778081,16,9/9/2024 9:59,9/9/2024 10:15,3075,34.042110,-118.256187,4672,34.045021,-118.253189,25555,30.0,One Way,Monthly Pass,electric,NaN,NaN
2830778,434777949,14,9/9/2024 9:59,9/9/2024 10:13,3019,34.038609,-118.260857,4273,34.025860,-118.284103,13614,30.0,One Way,Monthly Pass,standard,NaN,NaN
2830779,434771834,3,9/9/2024 9:59,9/9/2024 10:02,4576,34.022579,-118.405510,4563,34.017448,-118.409569,24052,365.0,One Way,Annual Pass,electric,NaN,NaN


In [ ]:
# Homogenized dataset for Capital
df_capital

,duration,start_time,end_time,start_station_id,start_station_name,end_station_id,end_station_name,bike_id,member_type,trip_id,bike_type,start_lat,start_lon,end_lat,end_lon
0,166.0,2016-01-01 00:06:58,2016-01-01 00:09:44,31102.0,11th & Kenyon St NW,31105.0,14th & Harvard St NW,W01346,Member,NaN,NaN,NaN,NaN,NaN,NaN
1,448.0,2016-01-01 00:10:20,2016-01-01 00:17:48,32039.0,Old Georgetown Rd & Southwick St,32002.0,Bethesda Ave & Arlington Rd,W22202,Member,NaN,NaN,NaN,NaN,NaN,NaN
2,715.0,2016-01-01 00:13:52,2016-01-01 00:25:48,31222.0,New York Ave & 15th St NW,31214.0,17th & Corcoran St NW,W21427,Member,NaN,NaN,NaN,NaN,NaN,NaN
3,213.0,2016-01-01 00:15:29,2016-01-01 00:19:03,31506.0,1st & Rhode Island Ave NW,31509.0,New Jersey Ave & R St NW,W01294,Member,NaN,NaN,NaN,NaN,NaN,NaN
4,872.0,2016-01-01 00:16:16,2016-01-01 00:30:49,31041.0,Prince St & Union St,31048.0,King St Metro South,W22058,Member,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37788030,NaN,2025-09-30 23:52:54.178,2025-09-30 23:58:47.132,31137.0,Park Pl & Rock Creek Church Rd NW,31120.0,10th St & Florida Ave NW,0000_electric,casual,CEE98DD5ED9E6B63,electric_bike,38.937135,-77.019125,38.920387,-77.025672
37788031,NaN,2025-09-30 23:53:08.457,2025-09-30 23:53:23.185,NaN,NaN,NaN,NaN,0000_electric,casual,8EA5A09E036B6B4C,electric_bike,38.990000,-77.090000,38.990000,-77.090000
37788032,NaN,2025-09-30 23:53:28.547,2025-09-30 23:58:51.329,31311.0,13th & O St NW/ Logan Circle,31223.0,Convention Center / 7th & M St NW,0000_electric,casual,7AA3E55FF072E5D0,electric_bike,38.908735,-77.029779,38.905737,-77.022270
37788033,NaN,2025-09-30 23:54:33.014,2025-09-30 23:58:43.502,31674.0,Maine Ave & Water St SW,31680.0,Half & I St SW,0000_electric,casual,5AE8D1849093DFEB,electric_bike,38.876722,-77.020973,38.879262,-77.011016


In [ ]:
# Save the homogenized datasets to CSV files in temporary storage
df_metro.to_csv(f"{temp_data_folder}/metro_dataset.csv", index=False)
df_capital.to_csv(f"{temp_data_folder}/capital_dataset.csv", index=False)

In [ ]:
df_metro = pd.read_csv(f"{temp_data_folder}/metro_dataset.csv")
df_capital = pd.read_csv(f"{temp_data_folder}/capital_dataset.csv")

## Stations 

In [ ]:
# Generate stations dataframe with all unique stations from both datasets
df_stations = generate_stations_dataframe(df_metro, df_capital)
df_stations

# Expected result:
# Generate bikes dataframe with all unique bikes from both datasets

,station_uuid,original_station_id,latitude,longitude,station_name,zip_code,city
0,cf45b489-b263-4441-8737-9adfe3ce991f,3000,34.025841,-118.422859,Virtual Station,90064,Los Angeles
1,114376bd-f807-4222-8af3-9d5a3fe00c16,3005,34.048500,-118.258537,7th & Flower,90017,Los Angeles
2,52a6916e-0f98-4bcb-bfbe-0aea100b9644,3006,34.045540,-118.256668,Olive & 8th,90017,Los Angeles
3,04d4dff3-2ea7-4cc3-915e-becbcb3941db,3007,34.050480,-118.254593,5th & Grand,90071,Los Angeles
4,a7b4a55b-7c53-44a7-a6f0-17678db93906,3008,34.046612,-118.262733,Figueroa & 9th,90017,Los Angeles
...,...,...,...,...,...,...,...
1305,ef9ff403-584c-4365-a431-b435cf369644,33000,38.904912,-77.058096,29th & M St NW,20037,Washington D.C.
1306,6780a258-13e5-4a60-a0c1-a186d27cd81b,33001,38.866105,-77.013645,2nd & U St SW,20260,Washington D.C.
1307,8486d927-23c5-4eda-ac21-0d96446a1bfc,33101,38.921583,-77.038564,17th St & Kalorama Rd NW,20009,Washington D.C.
1308,9f692754-0bbd-49a2-b64e-77c0c346dbe1,33200,38.896604,-77.039778,17th St & New York Ave NW,20006,Washington D.C.


In [ ]:
# Save stations dataframe to CSV file in temporary storage
df_stations.to_csv(f"{temp_data_folder}/stations_unmapped.csv", index=False)

In [ ]:
df_stations = pd.read_csv(f"{temp_data_folder}/stations_unmapped.csv")

## Bikes

In [ ]:
# Generate bikes dataframe with all unique bikes from both datasets
df_bikes = generate_bikes_dataframe(df_metro, df_capital)
df_bikes
# Expected result:
# All unique bikes from both datasets combined into a single dataframe

,bike_uuid,original_bike_id,bike_type
0,43541e9a-7db3-449d-ab62-75b5c1a44e1a,04823,standard
1,dae9dfd5-cc44-4fbc-965e-7b0936d64816,05006,standard
2,d7e9131d-a536-42c8-a313-1e93a9745fcd,05027,standard
3,134d7827-33da-42b8-8c18-9318884120d3,05715,standard
4,a23af8c6-4e64-4704-b0ec-5270800ee10e,05716,standard
...,...,...,...
11696,a2bd622e-2451-4bf7-a85a-69577a177629,w00583,standard
11697,ee669354-6aea-41ff-8e17-50c24d97a09d,w00644,standard
11698,f1d8ff00-d183-4ba9-aede-b3cb229dac63,w00765,standard
11699,92b65c02-fd4e-41ed-b0a0-0769839d23c1,w01100,standard


In [ ]:
# Save bikes dataframe to CSV file in temporary storage
df_bikes.to_csv(f"{temp_data_folder}/bikes.csv", index=False)

In [ ]:
df_bikes = pd.read_csv(f"{temp_data_folder}/bikes.csv")

## Trips

In [ ]:
# Generate trips dataframe combining both datasets
df_trips = generate_trips_dataframe(df_metro, df_capital)
df_trips
# Expected result:
# All trips from both datasets combined into a single dataframe

,trip_uuid,original_trip_id,start_time,end_time,start_station_id,end_station_id,bike_id,member_type,duration,plan_duration,trip_route_category
0,32d4029e-0142-4259-98f5-8cb6c1863adf,NaN,2016-01-01 00:06:58.000,2016-01-01 00:09:44.000,31102,31105,W01346,member,166.0,NaN,NaN
1,187de183-6194-4c2e-b001-8661800460bb,NaN,2016-01-01 00:10:20.000,2016-01-01 00:17:48.000,32039,32002,W22202,member,448.0,NaN,NaN
2,28c2dbab-fa66-4f40-8518-30338340d766,NaN,2016-01-01 00:13:52.000,2016-01-01 00:25:48.000,31222,31214,W21427,member,715.0,NaN,NaN
3,f6ffceaa-0b62-4f5a-be1c-72dcff31d448,NaN,2016-01-01 00:15:29.000,2016-01-01 00:19:03.000,31506,31509,W01294,member,213.0,NaN,NaN
4,36e752c0-0fe0-47d0-ad14-d4f445188400,NaN,2016-01-01 00:16:16.000,2016-01-01 00:30:49.000,31041,31048,W22058,member,872.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
35722411,781f142e-5dfa-43a9-83df-b44975587f1e,537A4C4D55A133CF,2025-09-30 23:52:22.258,2025-09-30 23:57:17.259,31232,31251,0000_classic,member,NaN,NaN,NaN
35722412,a9134fcb-85c6-4312-af29-257a313234e1,CEE98DD5ED9E6B63,2025-09-30 23:52:54.178,2025-09-30 23:58:47.132,31137,31120,0000_electric,casual,NaN,NaN,NaN
35722413,8967a276-8fa0-406f-acab-37c7276fe26a,7AA3E55FF072E5D0,2025-09-30 23:53:28.547,2025-09-30 23:58:51.329,31311,31223,0000_electric,casual,NaN,NaN,NaN
35722414,bd008339-eb76-4c26-875d-d8f36f95bca3,5AE8D1849093DFEB,2025-09-30 23:54:33.014,2025-09-30 23:58:43.502,31674,31680,0000_electric,casual,NaN,NaN,NaN


In [ ]:
# Save trips dataframe to CSV file in temporary storage
df_trips.to_csv(f"{temp_data_folder}/trips_unmapped.csv", index=False)

In [ ]:
df_trips = pd.read_csv(f"{temp_data_folder}/trips_unmapped.csv")

## Demographics

In [ ]:
# For this step, please set the environment variable of 'US_CENSUS_API_KEY' to get access to your data
# Generate demographics dataframe based on the stations' zip codes
df_demographics = generate_demographics_dataframe(df_stations)
df_demographics

# Expected result:
# Demographics dataframe with data for each zip code corresponding to the stations

Error fetching data: Expecting value: line 1 column 1 (char 0)
No information for zip code: 22081
Error fetching data: Expecting value: line 1 column 1 (char 0)
No information for zip code: 20376
Error fetching data: Expecting value: line 1 column 1 (char 0)
No information for zip code: 20189
Hidden values in column: median_household_income
Hidden values in column: median_age_female
Hidden values in column: median_age_male
Hidden values in column: median_age


,zip_code_uuid,zip_code,median_household_income,household_income_200000_to_more,household_income_150000_to_199999,household_income_125000_to_149999,household_income_100000_to_124999,household_income_75000_to_99999,household_income_60000_to_74999,household_income_50000_to_59999,...,population_two_or_more_races,population_other_race,population_native_hawaiian_and_other_pacific_islander,population_asian,population_american_indian_and_alaska_native,population_black,population_white,population_female,population_male,population
0,eb9ec3ab-ff35-49fa-ac3c-8f87b9dadb25,90064,129703.000000,4198.0,1187.0,678.0,1125.0,1162.0,826.0,384.0,...,3656.0,1497.0,64.0,5010.0,46.0,986.0,15235.0,13240.0,13254.0,26494.0
1,9effb671-1e16-4624-8de1-9dbd2755fd43,90017,51317.000000,956.0,675.0,688.0,846.0,1739.0,1382.0,915.0,...,2644.0,11047.0,0.0,5312.0,328.0,3782.0,5868.0,14275.0,14706.0,28981.0
2,dba97bf4-eb18-4e31-a2bb-44bd4a377c5a,90071,105989.559951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,87.0,87.0,0.0,0.0,0.0,86.0,44.0,131.0,173.0,304.0
3,029e8760-fb3e-4b64-b0d9-d488ecfc0c79,90291,119043.000000,4348.0,1543.0,960.0,1212.0,1542.0,884.0,663.0,...,2946.0,1520.0,45.0,1503.0,211.0,1465.0,18048.0,12639.0,13099.0,25738.0
4,8f3ac9e9-e74f-4897-b7bb-6ff44278d36c,90014,33822.000000,751.0,263.0,281.0,454.0,481.0,301.0,230.0,...,1120.0,1074.0,18.0,2149.0,33.0,2081.0,3065.0,4192.0,5348.0,9540.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,1810779d-bf25-442e-943a-c0ee22470c68,20722,85750.000000,247.0,301.0,175.0,202.0,294.0,97.0,137.0,...,932.0,2208.0,0.0,191.0,222.0,1879.0,930.0,3240.0,3122.0,6362.0
142,6095fac0-ab93-4422-a2e0-40460ac9828d,20743,76332.000000,1056.0,1868.0,1228.0,1756.0,1986.0,1788.0,1268.0,...,1710.0,3738.0,0.0,409.0,154.0,31429.0,1999.0,20488.0,18951.0,39439.0
143,e77e8507-f0fb-46aa-9aba-17f371c0f3a6,22081,104227.000000,2301.0,1249.0,854.0,1147.0,1319.0,852.0,534.0,...,2946.0,2034.0,8.0,2254.0,143.0,2465.0,11565.0,14786.0,14386.0,28981.0
144,2d286747-669a-42dc-bc69-3505876a9203,20376,104227.000000,2301.0,1249.0,854.0,1147.0,1319.0,852.0,534.0,...,2946.0,2034.0,8.0,2254.0,143.0,2465.0,11565.0,14786.0,14386.0,28981.0


In [ ]:
# Save demographics dataframe to CSV file in temporary storage
df_demographics.to_csv(f"{temp_data_folder}/demographics.csv", index=False)

In [ ]:
df_demographics = pd.read_csv(f"{temp_data_folder}/demographics.csv")

## Map foreign keys to UUIDs

In [ ]:
# Map stations foreign keys to UUIDs in trips dataframe
df_trips = map_trips_foreign_keys_to_uuids(df_trips, df_stations, df_bikes)
df_trips

# Expected result:
# Trips dataframe with station and bike foreign keys mapped to UUIDs (bike UUID, start station UUID, end station UUID)

,trip_uuid,original_trip_id,start_time,end_time,member_type,duration,plan_duration,trip_route_category,start_station_uuid,end_station_uuid,bike_uuid
0,32d4029e-0142-4259-98f5-8cb6c1863adf,NaN,2016-01-01 00:06:58.000,2016-01-01 00:09:44.000,member,166.0,NaN,NaN,d1736ab3-4527-4f19-9660-bcd74c7d861a,29e11a1a-7c59-436d-8325-c447bb83db1e,d294dff4-30d1-4b48-8e74-e88034e72a43
1,187de183-6194-4c2e-b001-8661800460bb,NaN,2016-01-01 00:10:20.000,2016-01-01 00:17:48.000,member,448.0,NaN,NaN,8a96fdfc-6816-4525-b2c0-09ad4e918745,1ba3378a-8166-4d10-a00c-8f1ebd637e37,0868589e-359d-4f84-a113-b770d09ce1a7
2,28c2dbab-fa66-4f40-8518-30338340d766,NaN,2016-01-01 00:13:52.000,2016-01-01 00:25:48.000,member,715.0,NaN,NaN,16a5a86a-9035-4103-88ae-857cbba10625,aefb75fa-abf1-49ca-b6eb-239a95504bd0,2a8c5688-282c-4b55-b1f7-30cf90a6fde2
3,f6ffceaa-0b62-4f5a-be1c-72dcff31d448,NaN,2016-01-01 00:15:29.000,2016-01-01 00:19:03.000,member,213.0,NaN,NaN,69dcc266-5f01-408b-87b1-4e0123be145b,2a20a566-4cd1-49be-ad16-8239490e3722,f4503de5-b237-44f5-8413-31eb046a1b65
4,36e752c0-0fe0-47d0-ad14-d4f445188400,NaN,2016-01-01 00:16:16.000,2016-01-01 00:30:49.000,member,872.0,NaN,NaN,f126c4d9-82db-48fc-963d-0145c5d500f7,bd0290f4-ef00-4193-a38b-26c1999ae822,63ff4839-4276-4996-b08c-2ef3e9f03a80
...,...,...,...,...,...,...,...,...,...,...,...
35722411,781f142e-5dfa-43a9-83df-b44975587f1e,537A4C4D55A133CF,2025-09-30 23:52:22.258,2025-09-30 23:57:17.259,member,NaN,NaN,NaN,5f594732-ce7f-403b-87d5-effc5649b15f,91a50297-81c6-4fc3-9046-8dd55a5e31e5,6624a3cf-4f13-459d-bf5a-d24d0c72dac9
35722412,a9134fcb-85c6-4312-af29-257a313234e1,CEE98DD5ED9E6B63,2025-09-30 23:52:54.178,2025-09-30 23:58:47.132,casual,NaN,NaN,NaN,97e21df6-899f-497d-a555-4adda48e284e,006a0d3c-b39b-41e2-a3f4-12c3b75fec3d,404da7cf-166e-4f2f-b306-00ddf7f4932b
35722413,8967a276-8fa0-406f-acab-37c7276fe26a,7AA3E55FF072E5D0,2025-09-30 23:53:28.547,2025-09-30 23:58:51.329,casual,NaN,NaN,NaN,a95726f3-47a8-4c84-9098-011f8559f43d,a05d68c8-f539-4f9d-a337-a16d5388d7da,404da7cf-166e-4f2f-b306-00ddf7f4932b
35722414,bd008339-eb76-4c26-875d-d8f36f95bca3,5AE8D1849093DFEB,2025-09-30 23:54:33.014,2025-09-30 23:58:43.502,casual,NaN,NaN,NaN,f6b74103-e9a6-4325-b308-6da411522169,fa7a0160-e079-4fe4-98a7-6826f077a371,404da7cf-166e-4f2f-b306-00ddf7f4932b


In [ ]:
# Save mapped trips dataframe to CSV file in temporary storage
df_trips.to_csv(f"{temp_data_folder}/trips.csv", index=False)

In [ ]:
df_trips = pd.read_csv(f"{temp_data_folder}/trips.csv")

In [ ]:
# Map stations foreign keys to UUIDs in stations dataframe
df_stations = map_stations_foreign_keys_to_uuids(df_stations, df_demographics)
df_stations

# Expected result:
# Stations dataframe with demographics foreign keys mapped to UUIDs (zip code UUID)

,station_uuid,original_station_id,latitude,longitude,station_name,city,zip_code_uuid
0,cf45b489-b263-4441-8737-9adfe3ce991f,3000,34.025841,-118.422859,Virtual Station,Los Angeles,eb9ec3ab-ff35-49fa-ac3c-8f87b9dadb25
1,114376bd-f807-4222-8af3-9d5a3fe00c16,3005,34.048500,-118.258537,7th & Flower,Los Angeles,9effb671-1e16-4624-8de1-9dbd2755fd43
2,52a6916e-0f98-4bcb-bfbe-0aea100b9644,3006,34.045540,-118.256668,Olive & 8th,Los Angeles,9effb671-1e16-4624-8de1-9dbd2755fd43
3,04d4dff3-2ea7-4cc3-915e-becbcb3941db,3007,34.050480,-118.254593,5th & Grand,Los Angeles,dba97bf4-eb18-4e31-a2bb-44bd4a377c5a
4,a7b4a55b-7c53-44a7-a6f0-17678db93906,3008,34.046612,-118.262733,Figueroa & 9th,Los Angeles,9effb671-1e16-4624-8de1-9dbd2755fd43
...,...,...,...,...,...,...,...
1305,ef9ff403-584c-4365-a431-b435cf369644,33000,38.904912,-77.058096,29th & M St NW,Washington D.C.,068c2216-5810-4832-b6d4-6ea32d3c5ab2
1306,6780a258-13e5-4a60-a0c1-a186d27cd81b,33001,38.866105,-77.013645,2nd & U St SW,Washington D.C.,b2486cb6-760b-4dcd-99cc-10434954a104
1307,8486d927-23c5-4eda-ac21-0d96446a1bfc,33101,38.921583,-77.038564,17th St & Kalorama Rd NW,Washington D.C.,bcc0987b-571e-41f5-b5ac-b29605a6005c
1308,9f692754-0bbd-49a2-b64e-77c0c346dbe1,33200,38.896604,-77.039778,17th St & New York Ave NW,Washington D.C.,caeea7c5-27b8-4fd5-afbe-31591614a169


In [ ]:
# Save mapped stations dataframe to CSV file in temporary storage
df_stations.to_csv(f"{temp_data_folder}/stations.csv", index=False)

In [55]:
df_stations = pd.read_csv(f"{temp_data_folder}/stations.csv")

# Feature Engineering

In [ ]:
# To transform all the datasets, please use automated_workflow.ps1 or run_pipeline.py
# Feature engineering on trips dataframe to include temporal and spatial features
df_trip_segment = df_trips.iloc[:100000] # only 100,000 records for fast computation

df_features = generate_trips_feature_engineering(df_trip_segment, df_stations)
df_features

# Expected result:
# Trips dataframe with additional engineered features such as trip duration category, distance, average speed, time of day, day of week, etc.

# Note: the records are fewer than the original because it discards records that have a speed higher than 50 km/hr

,trip_uuid,start_time,end_time,member_type,duration,start_station_uuid,end_station_uuid,bike_uuid,station_uuid_start,original_station_id_start,...,trip_duration_category,hour_of_day,trip_hour_category,day_of_week,is_weekend,month,season,distance_km,average_speed_kmh,trip_distance_category
0,32d4029e-0142-4259-98f5-8cb6c1863adf,2016-01-01 00:06:58,2016-01-01 00:09:44,member,166.0,d1736ab3-4527-4f19-9660-bcd74c7d861a,29e11a1a-7c59-436d-8325-c447bb83db1e,d294dff4-30d1-4b48-8e74-e88034e72a43,d1736ab3-4527-4f19-9660-bcd74c7d861a,31102,...,0,0,0,4,False,1,Winter,0.494579,10.725815,0
1,187de183-6194-4c2e-b001-8661800460bb,2016-01-01 00:10:20,2016-01-01 00:17:48,member,448.0,8a96fdfc-6816-4525-b2c0-09ad4e918745,1ba3378a-8166-4d10-a00c-8f1ebd637e37,0868589e-359d-4f84-a113-b770d09ce1a7,8a96fdfc-6816-4525-b2c0-09ad4e918745,32039,...,1,0,0,4,False,1,Winter,2.321930,18.658366,1
2,28c2dbab-fa66-4f40-8518-30338340d766,2016-01-01 00:13:52,2016-01-01 00:25:48,member,716.0,16a5a86a-9035-4103-88ae-857cbba10625,aefb75fa-abf1-49ca-b6eb-239a95504bd0,2a8c5688-282c-4b55-b1f7-30cf90a6fde2,16a5a86a-9035-4103-88ae-857cbba10625,31222,...,2,0,0,4,False,1,Winter,1.512782,7.606166,0
3,f6ffceaa-0b62-4f5a-be1c-72dcff31d448,2016-01-01 00:15:29,2016-01-01 00:19:03,member,214.0,69dcc266-5f01-408b-87b1-4e0123be145b,2a20a566-4cd1-49be-ad16-8239490e3722,f4503de5-b237-44f5-8413-31eb046a1b65,69dcc266-5f01-408b-87b1-4e0123be145b,31506,...,0,0,0,4,False,1,Winter,0.538232,9.054378,0
4,36e752c0-0fe0-47d0-ad14-d4f445188400,2016-01-01 00:16:16,2016-01-01 00:30:49,member,873.0,f126c4d9-82db-48fc-963d-0145c5d500f7,bd0290f4-ef00-4193-a38b-26c1999ae822,63ff4839-4276-4996-b08c-2ef3e9f03a80,f126c4d9-82db-48fc-963d-0145c5d500f7,31041,...,2,0,0,4,False,1,Winter,1.790658,7.384158,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0c83ca05-d98f-476b-b6c4-f70153072d6c,2016-01-20 07:45:51,2016-01-20 07:49:30,member,219.0,4c1939c5-5ec8-400a-b218-a7f129bd7764,ede0e622-687b-421c-98f3-ed6b813e634b,d5dc323b-027f-4b91-b43c-cacb9fa0cd79,4c1939c5-5ec8-400a-b218-a7f129bd7764,31201,...,0,7,1,2,False,1,Winter,0.705810,11.602351,0
99996,7ae95d63-dfee-4554-9678-952a70fa95c1,2016-01-20 07:45:53,2016-01-20 07:47:58,member,125.0,4b36e2c3-b5e7-4a8e-b1a0-67dbc0503235,020cebdc-9bc2-4fed-81b1-6849f635da23,77af9766-6450-4efb-afb6-1f56601d054c,4b36e2c3-b5e7-4a8e-b1a0-67dbc0503235,31313,...,0,7,1,2,False,1,Winter,0.410814,11.831430,0
99997,1c546c9e-dd5e-422d-b0b6-db5c36a4c9fb,2016-01-20 07:45:55,2016-01-20 07:49:20,member,205.0,406dff51-b78d-4dda-87f8-e97325934b55,23b9ca09-b441-4c65-adac-18575e0ada04,dd821c0d-98ea-4db2-9fac-ac4763773ff9,406dff51-b78d-4dda-87f8-e97325934b55,31229,...,0,7,1,2,False,1,Winter,0.544557,9.562948,0
99998,954877ae-705f-41e8-9fd9-4c4d3a74fcc0,2016-01-20 07:46:01,2016-01-20 07:56:01,member,600.0,a05d68c8-f539-4f9d-a337-a16d5388d7da,95c3488c-bd59-4938-a3c5-c1ab28102b16,b1fc14f9-c141-4a01-b3fc-3b73da2d6926,a05d68c8-f539-4f9d-a337-a16d5388d7da,31223,...,2,7,1,2,False,1,Winter,1.539407,9.236440,0
